In [33]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr, linregress
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

pio.renderers.default = "notebook_connected+plotly_mimetype"

# Load CSVs — adjust paths if needed
bb_df = pd.read_csv("WB_FINDEX_WIDEF.csv", encoding='utf-8', encoding_errors='ignore')
fi_df = pd.read_csv("WB_FINDEX_ACCOUNT_T_D_WIDEF.csv", encoding='utf-8', encoding_errors='ignore')

print("Mobile broadband raw data shape:", bb_df.shape)
print("Financial inclusion (Findex) raw data shape:", fi_df.shape)


In [34]:
# Normalize column names
fi_df.columns = fi_df.columns.str.strip()

# Filter for account‑ownership indicator
fi_sub = fi_df[fi_df['INDICATOR'] == "WB_FINDEX_ACCOUNT_T_D"].copy()

# Keep only real countries (3‑letter ISO codes) to exclude aggregates/regions
fi_sub = fi_sub[fi_sub['REF_AREA'].str.len() == 3]

# Define year columns present in dataset
year_cols = ['2011','2014','2017','2021','2022','2024']

# Melt from wide → long
fi_long = fi_sub.melt(
    id_vars=['REF_AREA','SEX','AGE','URBANISATION'],
    value_vars=year_cols,
    var_name='year',
    value_name='account_ownership_pct'
)

# Filter: adults 15+ (AGE == 'Y_GE15') and total urbanisation (URBANISATION == '_T')
fi_clean = fi_long[(fi_long['AGE'] == 'Y_GE15') & (fi_long['URBANISATION'] == '_T')].copy()

# Convert year to int, rename country code
fi_clean['year'] = fi_clean['year'].astype(int)
fi_clean = fi_clean.rename(columns={'REF_AREA': 'country_code'})

# Select relevant columns
fi_clean = fi_clean[['country_code', 'year', 'account_ownership_pct']]

# Drop missing values
fi_clean = fi_clean.dropna(subset=['account_ownership_pct'])

print("Cleaned Financial Inclusion data — sample:")
display(fi_clean.head())
print("Total rows after clean:", len(fi_clean))


Cleaned Financial Inclusion data — sample:


,country_code,year,account_ownership_pct
0,MEX,2011,13.400245
1,MKD,2011,82.886250
2,UGA,2011,3.493182
3,PHL,2011,23.217293
4,COG,2011,12.514530


Total rows after clean: 6674


In [36]:
# Strip column names
bb_df.columns = bb_df.columns.str.strip()

# Keep only countries (3-letter ISO codes)
bb_sub = bb_df[bb_df['REF_AREA'].str.len() == 3].copy()

# Detect year columns automatically
year_cols = [col for col in bb_sub.columns if col.isdigit()]

print("Detected mobile broadband year columns:", year_cols)

# Melt wide → long
bb_long = bb_sub.melt(
    id_vars=['REF_AREA','INDICATOR'],
    value_vars=year_cols,
    var_name='year',
    value_name='mobile_subs_per_100'
)

# Clean column names
bb_long['year'] = bb_long['year'].astype(int)
bb_long = bb_long.rename(columns={'REF_AREA':'country_code'})

# Keep relevant columns
bb_long = bb_long[['country_code','year','mobile_subs_per_100']].dropna()

bb_long.head()



Detected mobile broadband year columns: ['2011', '2014', '2017', '2021', '2022', '2024']


,country_code,year,mobile_subs_per_100
17,ARG,2011,3.800657
27,UKR,2011,47.348203
103,ARM,2011,6.966543
130,HRV,2011,39.883270
142,HRV,2011,33.169982


In [37]:
merged_df = pd.merge(bb_long, fi_clean, on=['country_code','year'], how='inner')

print("Merged dataset sample:")
merged_df.head()


Merged dataset sample:


,country_code,year,mobile_subs_per_100,account_ownership_pct
0,ARG,2011,3.800657,36.865283
1,ARG,2011,3.800657,28.272385
2,ARG,2011,3.800657,19.674924
3,ARG,2011,3.800657,26.812652
4,ARG,2011,3.800657,38.146205
